In [ ]:
import os
import argparse
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.io import read_image
from PIL import Image

from torchvision.models import resnet50, ResNet50_Weights

In [ ]:
60*5

In [ ]:
import time
time.sleep(2)
print("done")

In [ ]:
num_workers = 8
gpu = True
epochs = 1

In [ ]:
device = torch.device("mps")
device.type

In [ ]:
torch.cuda.is_available()

In [ ]:
class ResNet50Food(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = ResNet50_Weights.IMAGENET1K_V2
        self.pretrained = resnet50(weights=self.weights)
        self.fc2 = nn.Linear(in_features=1000, out_features=101, bias=True)
    
    def forward(self, x):
        x = self.pretrained(x)
        x = self.fc2(x)
        return x
    
net = ResNet50Food()
#net.train()

In [ ]:
net.parameters()

In [ ]:
for p in net.pretrained.parameters():
    print(p)

In [ ]:
for p in net.pretrained.parameters():
    p.requires_grad = False

In [ ]:
for p in net.pretrained.parameters():
    print(p)

In [ ]:
net2 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

In [ ]:
net2.fc = nn.Linear(in_features=2048, out_features=101, bias=True)

In [ ]:
net2.fc

In [ ]:
transform = transforms.Compose(
        [transforms.Resize(256,antialias=True),
        transforms.CenterCrop(224),
        #transforms.ConvertImageDtype(torch.float32),
        transforms.ToTensor(),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

preprocess = net.weights.transforms()

data = torchvision.datasets.Food101(root="./data",
                                    split="train",
                                    transform=preprocess
                                    )
train_data, val_data = torch.utils.data.random_split(data, [.8,.2])

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=num_workers)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False, num_workers=num_workers)

In [ ]:
X, y = next(iter(train_dataloader))
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
y

In [ ]:
#preprocess = net.weights.transforms()
preprocess = ResNet50_Weights.IMAGENET1K_V2.transforms()

In [ ]:
img_transformed = preprocess(X)

In [ ]:
def training_loop(net, trainloader, valloader, gpu=False, epochs=1):
    if gpu == False:
        device = torch.device("cpu")
    elif gpu == True:
        device = torch.device("mps")
    net = net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(epochs): 
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = net(inputs)
            train_loss = criterion(outputs, labels)
            train_loss.backward()
            optimizer.step()
            
            # print statistics
            #print(f"{train_loss.item()}")
            running_loss += train_loss.item()
            print(train_loss.item())
            if i % 100 == 99:  # print every 200 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.8f}')
                running_loss = 0.0
        #train_acc = accuracy_score(net, trainloader, gpu=gpu)
        #val_acc = accuracy_score(net, valloader, gpu=gpu)
        #print(f"Epoch {epoch} train_acc: {train_acc}, val_acc: {val_acc}")
    PATH = './models/net.pth'
    torch.save(net.state_dict(), PATH)
    print(f'Training complete - model saved to {PATH}')

In [ ]:
training_loop(net2, train_dataloader, val_dataloader, gpu=gpu, epochs=epochs)

In [ ]:
PATH = './models/net2.pth'
torch.save(net2.state_dict(), PATH)

In [ ]:
net2 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
net2.fc = nn.Linear(in_features=2048, out_features=101, bias=True)
net2.load_state_dict(torch.load(PATH))

In [ ]:
net2.fc

In [ ]:
def resnet():
    #net = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
    net = resnet50()
    net.fc = nn.Linear(in_features=2048, out_features=101, bias=True)
    return net

In [ ]:
net = resnet()

In [ ]:
#torch.save(net.state_dict(), PATH)

In [ ]:
model = 'ResNet'

net = resnet()
PATH = f"./models/{model}.pth"
net.load_state_dict(torch.load(PATH))

In [ ]:
net.load_state_dict(torch.load(PATH))

In [ ]:
for x in torch.load(PATH):
    print(x)